In [ ]:
import logging
import azure.functions as func
import requests
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import datetime
import time

APP_ID = "92c0a55a"
APP_KEY = "d58f307a1667e2e502b8da4ad83ef81c"

COUNTRY = "au"
QUERIES = ["data scientist", "software engineer", "cybersecurity", "ai engineer"]
PAGES_PER_ROLE = 2

SQL_SERVER = "auseast.database.windows.net"
SQL_PORT = 1433
SQL_DATABASE = "JobMarketDb"
SQL_USERNAME = "CloudSAfc8188ca"
#Database password hidden
SQL_PASSWORD = "passwordishiddem"

CONN_STR = (
    f"mssql+pytds://{SQL_USERNAME}:{SQL_PASSWORD}"
    f"@{SQL_SERVER}:{SQL_PORT}/{SQL_DATABASE}"
    "?encrypt=yes&trustservercertificate=no&hostNameInCertificate=*.database.windows.net"
)

app = func.FunctionApp()

@app.timer_trigger(
    schedule="0 0 0 * * *", 
    arg_name="myTimer",
    run_on_startup=True
)
def timer_trigger(myTimer: func.TimerRequest):

    logging.info("💼 Job ingestion function started.")

    #Fetching job data
    all_rows = []
    for role in QUERIES:
        for page in range(1, PAGES_PER_ROLE + 1):

            url = f"https://api.adzuna.com/v1/api/jobs/{COUNTRY}/search/{page}"
            params = {
                "app_id": APP_ID,
                "app_key": APP_KEY,
                "what": role,
                "results_per_page": 50,
                "content-type": "application/json"
            }

            res = requests.get(url, params=params)
            data = res.json()

            for job in data.get("results", []):
                all_rows.append({
                    "role": role,
                    "title": job["title"],
                    "company": job.get("company", {}).get("display_name"),
                    "location": job.get("location", {}).get("display_name"),
                    "created": job["created"],
                    "salary_min": job.get("salary_min"),
                    "salary_max": job.get("salary_max"),
                    "description": job.get("description", "")[:500],
                    "fetched_at": datetime.utcnow()
                })

            time.sleep(1)

    df = pd.DataFrame(all_rows)
    logging.info(f"Downloaded {len(df)} job listings")

    df["created"] = pd.to_datetime(df["created"], utc=True)
    df["salary_min"] = df.groupby("role")["salary_min"].transform(lambda x: x.fillna(x.median()))
    df["salary_max"] = df.groupby("role")["salary_max"].transform(lambda x: x.fillna(x.median()))
    df["salary_avg"] = df[["salary_min", "salary_max"]].mean(axis=1)

    logging.info("Data cleaning complete")

    try:
        engine = create_engine(CONN_STR, connect_args={"autocommit": True})

        insert_sql = text("""
            INSERT INTO JobPosts (
                role, title, company, location, created,
                salary_min, salary_max, salary_avg, description, fetched_at
            )
            VALUES (
                :role, :title, :company, :location, :created,
                :salary_min, :salary_max, :salary_avg, :description, :fetched_at
            )
        """)

        allowed_cols = [
            "role", "title", "company", "location", "created",
            "salary_min", "salary_max", "salary_avg", "description", "fetched_at"
        ]

        rows = df[allowed_cols].to_dict(orient="records")

        with engine.begin() as conn:
            for row in rows:
                conn.execute(insert_sql, row)

        logging.info("✅ Uploaded all jobs into Azure SQL successfully")

    except Exception as e:
        logging.error(f"SQL Insert failed: {e}")
        return

    logging.info("Job ingestion pipeline finished.")
